In [2]:
!pip install datasets transformers nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.2/431.2 kB 2.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 31.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.8 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.0/770.0 kB 7.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 23.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3

In [3]:
import os
import os.path as osp
import nltk
import random
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords
# english_stopwords = stopwords.words("english")
import numpy as np
import re
import seaborn as sns
sns.set_theme(style="whitegrid")
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd
import string
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

import datasets
from datasets import load_dataset
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
import multiprocessing
import time 

from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Matplotlib is building the font cache; this may take a moment.


In [4]:
dev_file = "/efs/IS/dev_data.csv"
val_file = "/efs/IS/val_data.csv"
train_file = "/efs/IS/train_data.csv"

In [5]:
pretrained_model = "bert-base-uncased"
batch_size = 64
max_para_length = 128

In [6]:
#load csv files
dataset = load_dataset("csv", 
                       data_files={"train": "train_data.csv",
                                   "val": "val_data.csv",
                                   "test": "dev_data.csv"})
dataset

Using custom data configuration default-4bb65942c3871d79


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-4bb65942c3871d79/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['para', 'label'],
        num_rows: 44195
    })
    val: Dataset({
        features: ['para', 'label'],
        num_rows: 9368
    })
    test: Dataset({
        features: ['para', 'label'],
        num_rows: 12871
    })
})

In [7]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model, do_lower=True)

def preprocess(examples):
    return tokenizer(examples["para"], truncation=True, 
                     padding="max_length", max_length=max_para_length,
                     return_token_type_ids=False)

n_cpu = multiprocessing.cpu_count()

#preprocess
dataset = dataset.map(preprocess,# preprocessing function
                      num_proc=n_cpu,# number of workers to use
                      remove_columns=["para"] # columns to remove after preprocessing
                     )
dataset

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

#0:   0%|          | 0/11049 [00:00<?, ?ex/s]

#1:   0%|          | 0/11049 [00:00<?, ?ex/s]

...

#0:   0%|          | 0/3218 [00:00<?, ?ex/s]

#1:   0%|          | 0/3218 [00:00<?, ?ex/s]

#2:   0%|          | 0/3218 [00:00<?, ?ex/s]

#3:   0%|          | 0/3217 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 44195
    })
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 9368
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 12871
    })
})

In [8]:
dataset.set_format(type='torch', columns=['label', 'input_ids', 'attention_mask'])
    
train_args = dict(shuffle=True, batch_size=batch_size, num_workers=n_cpu, pin_memory=True, drop_last=False) 
train_loader = torch.utils.data.DataLoader(dataset["train"], **train_args)

val_args = dict(shuffle=False, batch_size=batch_size, num_workers=n_cpu, pin_memory=True, drop_last=False) 
val_loader = torch.utils.data.DataLoader(dataset["val"], **val_args)


## First: Fixed Bert word Embeddings as input BiLSTM encoder

In [9]:
class BertEmbedding(nn.Module):
  def __init__(self, pretrained_model):
    super().__init__()
    self.model = BertModel.from_pretrained(pretrained_model, output_hidden_states = True)

  def forward(self, x):
    outputs = self.model(x["input_ids"])
    hidden_states = outputs[2]
    embedding = torch.cat((hidden_states[-1],hidden_states[-2],hidden_states[-3],hidden_states[-4]), dim = 2)
    return embedding


class ParaEncoder(nn.Module):
  def __init__(self, bilayers = 1, input_dim = 3072, hidden_size = 512):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_size
    self.lstm = nn.LSTM(
            input_size=input_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=True, bidirectional=True)
    
    for name, param in self.lstm.named_parameters():
      if 'bias' in name:
        nn.init.constant(param, 0.0)
      elif 'weight' in name:
        nn.init.orthogonal(param)

  def forward(self, x): # (B, tokens, input_dim)
    outputs, _ = self.lstm(x) # (B, tokens, 2*hidden_dim)
    first = outputs[:, 0, self.hidden_dim:]
    second = outputs[:, -1, :self.hidden_dim]
    para_embed = torch.cat((second,first), dim = 1)
    return para_embed #(B, 2*hidden_dim)



class ParaDecoderLinear(nn.Module):
  def __init__(self, input_size, output_size = 1):
    super().__init__()
    self.linear = nn.Linear(input_size, 1, bias= True);
    # self.layers = nn.Sequential(nn.Linear(input_size, output_size, bias = True), 
    #                             nn.BatchNorm1d(output_size), 
    #                             nn.ReLU(inplace = True), 
    #                             nn.Linear(output_size, 1, bias = True))
    
    # for mod in self.modules():
    #   if isinstance(mod, nn.BatchNorm1d):
    #     nn.init.constant_(mod.weight.data, 1)
    #     if(mod.bias is not None):
    #       nn.init.constant_(mod.bias.data, 0)

  def forward(self, x): # (B, 2*hidden_dim)
    return self.linear(x) #(B,1)

class EncoderDecoder(nn.Module):
  def __init__(self, decoder_output_size = 1, encoder_bilayers = 1, encoder_input_dim = 3072, encoder_hidden_size = 512):
    super().__init__()
    self.para_encoder = ParaEncoder(bilayers = encoder_bilayers, input_dim = encoder_input_dim, hidden_size = encoder_hidden_size)
    self.para_decoder = ParaDecoderLinear(input_size = encoder_hidden_size*2).to(device)

  def forward(self, x): # (B, 2*hidden_dim)
    para_vec = self.para_encoder(x)
    return self.para_decoder(para_vec)

In [10]:
def train(embed_model, para_model, data_loader):
  embed_model.eval()
  para_model.train()
  
  avg_loss = []
  all_predictions = []
  all_targets = []
  start = time.time()

  for i, batch in enumerate(tqdm(data_loader, desc="Epoch", leave=False)):
    optimizer.zero_grad()

    targets = batch["label"].to(device)
    batch = {k: v.to(device) for k, v in batch.items() if k != "label"}
    
    with torch.no_grad():
      outputs = embed_model(batch)
    
    output = para_model(outputs)
    

    loss = criterion(torch.squeeze(output), targets.float())
    avg_loss.extend([loss.item()]*len(targets))

    output = nn.Sigmoid()(output)

    all_predictions.extend((output >= 0.5).tolist())
    all_targets.extend(targets.tolist())

    loss.backward()
    optimizer.step()
    
  end = time.time()
  avg_loss = np.mean(avg_loss)
  print('learning_rate: {}'.format(scheduler.get_lr()))
  print('Training loss: {:.2f}, Time: {}'.format(avg_loss, end-start))
  
  all_predictions = np.array(all_predictions)
  all_targets = np.array(all_targets)
  scores = precision_recall_fscore_support(all_targets, all_predictions, 
                                            average="weighted", zero_division=0.)
  
  test_scores={
      "eval_accuracy": (all_predictions == all_targets).sum() / len(all_predictions),
      "eval_precision": scores[0],
      "eval_recall": scores[1],
      "eval_f-1": scores[2]
  }
  print(test_scores)
  return test_scores["eval_f-1"]


In [11]:
def validate(embed_model, para_model, data_loader):
  embed_model.eval()
  para_model.eval()

  avg_loss = []
  all_predictions = []
  all_targets = []
  start = time.time()

  for i, batch in enumerate(tqdm(data_loader, desc="Epoch", leave=False)):
    optimizer.zero_grad()

    targets = batch["label"].to(device)
    batch = {k: v.to(device) for k, v in batch.items() if k != "label"}
    
    with torch.no_grad():
      outputs = embed_model(batch)
  
      output = para_model(outputs)

      loss = criterion(torch.squeeze(output), targets.float())
      avg_loss.extend([loss.item()]*len(targets))

      output = nn.Sigmoid()(output)

      all_predictions.extend((output >= 0.5).tolist())
      all_targets.extend(targets.tolist())

    
  end = time.time()
  avg_loss = np.mean(avg_loss)
  print('learning_rate: {}'.format(scheduler.get_lr()))
  print('Validation loss: {:.2f}, Time: {}'.format(avg_loss, end-start))
  
  all_predictions = np.array(all_predictions)
  all_targets = np.array(all_targets)
  scores = precision_recall_fscore_support(all_targets, all_predictions, 
                                            average="weighted", zero_division=0.)
  
  test_scores={
      "eval_accuracy": (all_predictions == all_targets).sum() / len(all_predictions),
      "eval_precision": scores[0],
      "eval_recall": scores[1],
      "eval_f-1": scores[2]
  }
  print(test_scores)
  return test_scores["eval_f-1"]


In [12]:
def save(model, acc, best=""):
    if not os.path.exists('./bert_base_linear/'):
        os.mkdir('./bert_base_linear/')

    torch.save(model.state_dict(), './bert_base_linear/'+'/{}model_params_{}.pth'.format(best, acc))
    

In [13]:
embed_model = BertEmbedding(pretrained_model).to(device)
model = EncoderDecoder().to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_24157/2524482474.py:26: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(param)
/tmp/

In [14]:
epochs = 35
lamda = 5e-5  #L2 regularization
learning_rate = 1e-3

criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('efs/hw3_checkpoint/4/best_loss_optimizer_model_params_00000046.pth'))    

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i for i in range(3,35,5)], gamma=0.75)


In [15]:
best_val_f1 = 0
for epoch in range(epochs):
  print('Epoch #{}'.format(epoch+1))
  
  train_f1 = train(embed_model, model, train_loader)
  val_f1 = validate(embed_model, model, val_loader)
  
  if val_f1 > best_val_f1:
    best_val_f1 = val_f1
    save(model, best_val_f1, best = "model_")
    save(optimizer, best_val_f1, best = "optimizer_")

  scheduler.step()

Epoch #1


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:418: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


learning_rate: [0.001]
Training loss: 0.21, Time: 457.4597535133362
{'eval_accuracy': 27969.52562507071, 'eval_precision': 0.9054664508363292, 'eval_recall': 0.9065957687521213, 'eval_f-1': 0.9059011198577562}


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:418: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


learning_rate: [0.001]
Validation loss: 0.22, Time: 73.26233077049255
{'eval_accuracy': 5734.138343296328, 'eval_precision': 0.9020839346278229, 'eval_recall': 0.9036080273270709, 'eval_f-1': 0.9022427006267076}
Epoch #2


...

learning_rate: [0.00010011291503906251]
Validation loss: 0.37, Time: 75.00900220870972
{'eval_accuracy': 5496.966695132365, 'eval_precision': 0.9282989154131268, 'eval_recall': 0.923462852263023, 'eval_f-1': 0.9247411465533508}
Epoch #35


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:418: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


learning_rate: [0.00013348388671875002]
Training loss: 0.05, Time: 421.14172101020813
{'eval_accuracy': 27691.769091526192, 'eval_precision': 0.9824039923183996, 'eval_recall': 0.9823509446769997, 'eval_f-1': 0.9823730065004419}


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:418: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


learning_rate: [0.00013348388671875002]
Validation loss: 0.37, Time: 70.12679672241211
{'eval_accuracy': 5556.034158838599, 'eval_precision': 0.9252574153872763, 'eval_recall': 0.922929120409906, 'eval_f-1': 0.9237066251117337}
